In [3]:
from datetime import datetime
import pandas as pd
from geopy import distance
import numpy as np

import pymongo
from pymongo import MongoClient

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

In [4]:
client = MongoClient('localhost', 27017)

db = client['avl_pipeline_test']
in_coll = db['trips']
edge_coll = db['edges']
out_coll = db['edges_standardized']

In [5]:
# Optional - Clean the output collection!
out_coll.delete_many({});

In [6]:
def normalize_duration(duration, sequence):
    
    edge_deets = edge_coll.find_one({'sequence': sequence})
    
    return (duration - edge_deets['mean'])/edge_deets['std_dev']

In [7]:
for example in in_coll.find():

    trip_id = example['_id']

    edge_sequence = list(example['edges'].keys())

    for idx, edge_seq in enumerate(edge_sequence):

        edge_info = example['edges'][edge_seq]

        edge_details = {}

        edge_details['edge_id'] = trip_id + "_" + edge_seq

        edge_details['sequence'] = edge_seq

        edge_details['weekend'] = example['weekend']
        edge_details['rush_hour'] = example['rush_hour']
        edge_details['midday'] = example['midday']

#         normed_edge = normalize_duration(edge_info['edge_time'], edge_seq)

#         example['edges'][edge_seq]['normed_edge'] = normed_edge
        edge_details['standard_edge'] = edge_info['edge_time']


        if idx == 0:

            edge_details['first_prior'] = 0
            edge_details['second_prior'] = 0

        elif idx == 1:

            frstp_seq = edge_sequence[idx-1]        
            edge_details['first_prior'] = example['edges'][frstp_seq]['edge_time']

            edge_details['second_prior'] = 0

        else:

            frstp_seq = edge_sequence[idx-1]        
            edge_details['first_prior'] = example['edges'][frstp_seq]['edge_time']

            scndp_seq = edge_sequence[idx-2]        
            edge_details['second_prior'] = example['edges'][scndp_seq]['edge_time']

        out_coll.insert_one(edge_details)

In [8]:
out_coll.count()

53561

In [15]:
edge_coll.find_one()

{'_id': ObjectId('5ae7b4ba3ad39e762bcbecfb'),
 'sequence': '1',
 'mean': 81.58061953931691,
 'std_dev': 38.841898301128815}

In [44]:
list(out_coll.find())

[{'_id': ObjectId('5ae7cc6a3ad39e78321c4419'),
  'edge_id': '7253717_2016-10-28_HKH3O_1',
  'sequence': '1',
  'weekend': 0,
  'rush_hour': 0,
  'midday': 0,
  'normed_edge': 0.1652694832501667,
  'first_prior': 0,
  'second_prior': 0},
 {'_id': ObjectId('5ae7cc6a3ad39e78321c441a'),
  'edge_id': '7253717_2016-10-28_HKH3O_2',
  'sequence': '2',
  'weekend': 0,
  'rush_hour': 0,
  'midday': 0,
  'normed_edge': -0.06165484088422265,
  'first_prior': 0.1652694832501667,
  'second_prior': 0},
 {'_id': ObjectId('5ae7cc6a3ad39e78321c441b'),
  'edge_id': '7253717_2016-10-28_HKH3O_3',
  'sequence': '3',
  'weekend': 0,
  'rush_hour': 0,
  'midday': 0,
  'normed_edge': -0.5429640499652788,
  'first_prior': -0.06165484088422265,
  'second_prior': 0.1652694832501667},
 {'_id': ObjectId('5ae7cc6a3ad39e78321c441c'),
  'edge_id': '7253717_2016-10-28_HKH3O_4',
  'sequence': '4',
  'weekend': 0,
  'rush_hour': 0,
  'midday': 0,
  'normed_edge': 0.3012911771091698,
  'first_prior': -0.5429640499652788,
